# 준비

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Embedding, GRU, Dense, Input, LSTM, Embedding, Dense, Masking

import nltk.translate.bleu_score as bleu

import pandas as pd
import numpy as np

import re
import os
import unicodedata

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.9.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 데이터

In [2]:
file_path = '../data/ex10/fra-eng/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')

DATA_SIZE = 30000
lines = lines.sample(DATA_SIZE)
lines = lines.reset_index().drop(["index"],axis=1)
print('샘플의 수 :',len(lines))
lines[:5]

샘플의 수 : 30000


,eng,fra,cc
0,I have to buy flowers for my girlfriend.,Je dois acheter des fleurs pour ma petite amie.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
1,Do what you see fit.,Fais ce que tu juges bon.,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Tom is unfazed.,Tom reste imperturbable.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,You're breaking my heart.,Tu me fends le cœur.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Why not just quit?,Pourquoi ne pas simplement cesser ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


## 전처리

In [3]:
def to_ascii(s):
    # 프랑스어 악센트(accent) 삭제
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [4]:
def preprocess_sentence(sent):
    sent = sent.lower() # 소문자
    sent = to_ascii(sent) # 악센트 삭제
    sent = re.sub(r"([?.!,¿])", r" \1", sent) # 구두점 사이 공백추가
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent) # 불필요한 문자 제거
    sent = re.sub(r"\s+", " ", sent) # 여러개 공백을 1개로
    return sent

In [5]:
# 전처리 샘플
preprocess_sentence(lines.loc[0,"fra"])

'je dois acheter des fleurs pour ma petite amie .'

In [6]:
# 전처리
eng_line = lines["eng"]
fra_line = lines["fra"]
eng_line = [preprocess_sentence(w).split() for w in eng_line] # encoder input
fra_line_in = [["<sos>"]+preprocess_sentence(w).split() for w in fra_line] # decoder input
fra_line_out = [preprocess_sentence(w).split()+["<eos>"] for w in fra_line] # decoder target

# del fra_line
# del lines

In [7]:
eng_line, fra_line, eng_line
print('인코더의 입력 :',eng_line[:5])
print('\n디코더의 입력 :',fra_line_in[:5])
print('\n디코더의 레이블 :',fra_line_out[:5])

인코더의 입력 : [['i', 'have', 'to', 'buy', 'flowers', 'for', 'my', 'girlfriend', '.'], ['do', 'what', 'you', 'see', 'fit', '.'], ['tom', 'is', 'unfazed', '.'], ['you', 're', 'breaking', 'my', 'heart', '.'], ['why', 'not', 'just', 'quit', '?']]

디코더의 입력 : [['<sos>', 'je', 'dois', 'acheter', 'des', 'fleurs', 'pour', 'ma', 'petite', 'amie', '.'], ['<sos>', 'fais', 'ce', 'que', 'tu', 'juges', 'bon', '.'], ['<sos>', 'tom', 'reste', 'imperturbable', '.'], ['<sos>', 'tu', 'me', 'fends', 'le', 'c', 'ur', '.'], ['<sos>', 'pourquoi', 'ne', 'pas', 'simplement', 'cesser', '?']]

디코더의 레이블 : [['je', 'dois', 'acheter', 'des', 'fleurs', 'pour', 'ma', 'petite', 'amie', '.', '<eos>'], ['fais', 'ce', 'que', 'tu', 'juges', 'bon', '.', '<eos>'], ['tom', 'reste', 'imperturbable', '.', '<eos>'], ['tu', 'me', 'fends', 'le', 'c', 'ur', '.', '<eos>'], ['pourquoi', 'ne', 'pas', 'simplement', 'cesser', '?', '<eos>']]


## 데이터 토큰화

In [8]:
tokenizer_eng = Tokenizer(filters="", lower=False)
tokenizer_eng.fit_on_texts(eng_line)
encoder_input = tokenizer_eng.texts_to_sequences(eng_line)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(fra_line_in)
tokenizer_fra.fit_on_texts(fra_line_out)

decoder_input = tokenizer_fra.texts_to_sequences(fra_line_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(fra_line_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [9]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (30000, 32)
디코더의 입력의 크기(shape) : (30000, 37)
디코더의 레이블의 크기(shape) : (30000, 37)


In [10]:
src_vocab_size = len(tokenizer_eng.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 7654, 프랑스어 단어 집합의 크기 : 11068


### 단어사전

In [11]:
src_to_index = tokenizer_eng.word_index
index_to_src = tokenizer_eng.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

# 훈련

## TTS

In [12]:
n_of_val = int(DATA_SIZE*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3000


In [13]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [14]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (27000, 32)
훈련 target 데이터의 크기 : (27000, 37)
훈련 target 레이블의 크기 : (27000, 37)
테스트 source 데이터의 크기 : (3000, 32)
테스트 target 데이터의 크기 : (3000, 37)
테스트 target 레이블의 크기 : (3000, 37)


In [15]:
print('훈련 source 데이터의 샘플 :',encoder_input_train[0])
print('훈련 target 데이터의 샘플 :',decoder_input_train[0])
print('훈련 target 레이블의 샘플 :',decoder_target_train[0])
print('테스트 source 데이터의 샘플 :',encoder_input_test[0])
print('테스트 target 데이터의 샘플 :',decoder_input_test[0])
print('테스트 target 레이블의 샘플 :',decoder_target_test[0])

훈련 source 데이터의 샘플 : [  2  20   4 196 546  25  26 490   1   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
훈련 target 데이터의 샘플 : [  2   4 137 271  40 520  31  86 353 390   1   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]
훈련 target 레이블의 샘플 : [  4 137 271  40 520  31  86 353 390   1   3   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]
테스트 source 데이터의 샘플 : [  2  33   4 706   4  49  17   3  25  23  68 198   1   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
테스트 target 데이터의 샘플 : [  2   4  48 870 116 652   5  10  97  31  18  49 467   6  57 958   1   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]
테스트 target 레이블의 샘플 : [  4  48 870 116 652   5  10  97  31  18  49 467   6  57 958   1   3   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]


## 번역 모델 생성

In [16]:
embedding_dim = 128
hidden_units = 128

In [22]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

es = EarlyStopping(monitor="val_loss",patience=5, restore_best_weights=True)
rl = ReduceLROnPlateau(factor=0.1,patience=3)

In [23]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=256, epochs=100,callbacks=[es,rl])

Epoch 1/100
106/106 [==============================] - 15s 87ms/step - loss: 3.3241 - acc: 0.7408 - val_loss: 1.5520 - val_acc: 0.7545 - lr: 0.0010
Epoch 2/100
106/106 [==============================] - 7s 67ms/step - loss: 1.4450 - acc: 0.7626 - val_loss: 1.3544 - val_acc: 0.7783 - lr: 0.0010
Epoch 3/100
106/106 [==============================] - 7s 68ms/step - loss: 1.3033 - acc: 0.8004 - val_loss: 1.2685 - val_acc: 0.8087 - lr: 0.0010
Epoch 4/100
106/106 [==============================] - 7s 67ms/step - loss: 1.2329 - acc: 0.8118 - val_loss: 1.2133 - val_acc: 0.8135 - lr: 0.0010
Epoch 5/100
106/106 [==============================] - 7s 68ms/step - loss: 1.1829 - acc: 0.8151 - val_loss: 1.1753 - val_acc: 0.8173 - lr: 0.0010
Epoch 6/100
106/106 [==============================] - 7s 68ms/step - loss: 1.1467 - acc: 0.8190 - val_loss: 1.1466 - val_acc: 0.8193 - lr: 0.0010
Epoch 7/100
106/106 [==============================] - 7s 68ms/step - loss: 1.1176 - acc: 0.8215 - val_loss: 1.1256 -

# 테스트

In [24]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [25]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq]+states_value,verbose=0)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' ' + sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [26]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if (encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence


# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if (encoded_word != 0 and encoded_word != tar_to_index['<sos>']
                and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
    return sentence

In [27]:
english = [] # 영어
french_ref = [] # 프랑스어 정답
french_cand = [] # 프랑스어 기계번역

In [28]:
for seq_index in range(500):
    try:
        input_seq = encoder_input_test[seq_index:seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)

        english.append(seq_to_src(encoder_input_test[seq_index]))
        french_ref.append(seq_to_tar(decoder_input_test[seq_index]))
        french_cand.append(decoded_sentence[1:-5])
    except:
        pass

1/1 [==============================] - 0s 13ms/step


In [29]:
len(english)

500

In [30]:
pd.DataFrame({"English":english, "French_cand":french_cand, "French_ref":french_ref}).to_csv("번역결과.csv", header=True, encoding="utf-8")

# 결과확인

In [31]:
result = pd.read_csv("번역결과.csv", index_col=0)
result

,English,French_cand,French_ref
0,i want to apologize to all of you for what jus...,je veux que tu m as de mal a mon travail a fai,je veux presenter mes excuses a vous tous pour...
1,this isn t sugar .,ce n etait pas cher .,ce n est pas du sucre .
2,she sent me an urgent telegram .,tom a dit qu elle est une petite amie .,elle m a envoye un telegramme urgent .
3,this is outlandish .,c est un .,c est etrange .
4,do you want me to be happy ?,voulez vous vraiment que je vous prie ?,voulez vous que je sois heureux ?
...,...,...,...
495,i told him to come the following day .,je lui ai dit a l ecole .,je lui ai dit de venir le jour suivant .
496,we waited in the movie theater for the film to...,nous avons eu des photos de la fete .,nous avons attendu le debut du film dans la sa...
497,there s a hotel across the street .,il y a un accident de l eau .,il y a un hotel de l autre cote de la rue .
498,we have to wait for him .,il se pourrait tous une bonne .,nous devons l attendre .


In [32]:
score_list = []
for idx, data in result.iterrows():
    temp = []
    temp2 = []
    a = data["French_ref"].split()
    temp.append(a)
    b = data["French_cand"].split()
    temp2.append(b)
    score = bleu.sentence_bleu(temp,b)
    score_list.append(score)

/home/komingsu/miniconda3/envs/tf/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/komingsu/miniconda3/envs/tf/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/komingsu/miniconda3/envs/tf/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider usi

In [33]:
sum(score_list)/len(score_list)

0.03026152282078364